In [1]:
import re
import os
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter


Using TensorFlow backend.


# Reading and Processing Data

In [2]:
file_source_path = './Health-Tweets'
all_files = os.listdir(file_source_path)

tweets = pd.DataFrame()
list_ = []

for file_ in all_files:
    df = pd.read_csv(file_source_path + '/' + file_, index_col=0,
                     sep='|', header=None, error_bad_lines=False,
                     encoding = "ISO-8859-1")
    df.columns = ['datetime', 'text']
    df['source'] = file_.replace('.txt', '')
    list_.append(df)

tweets = pd.concat(list_)
tweets['text'] = tweets['text'].str.split('http://', 1, expand=True)[0]
tweets['text'] = tweets['text'].str.lower()
tweets['text'] = tweets['text'] + '<EOS>'
tweets['text'] = tweets['text'].str.replace('&amp;', '&')
tweets['text'] = tweets['text'].str.replace('&gt;', '>')
tweets['text'] = tweets['text'].str.replace('&ls;', '<')
tweets['text'] = tweets['text'].str.replace('([$*\",\'%//\-@&:;!?()#])', r' \1 ')
tweets['text'] = tweets['text'].str.replace("(<EOS>)", r" \1 ")
tweets['text'] = tweets['text'].str.replace('\s{2,}', ' ')

tweets = tweets.head(10000)

b'Skipping line 846: expected 3 fields, saw 4\nSkipping line 904: expected 3 fields, saw 4\nSkipping line 914: expected 3 fields, saw 4\nSkipping line 1264: expected 3 fields, saw 4\nSkipping line 1269: expected 3 fields, saw 4\nSkipping line 1293: expected 3 fields, saw 4\nSkipping line 1348: expected 3 fields, saw 4\nSkipping line 1430: expected 3 fields, saw 4\nSkipping line 1486: expected 3 fields, saw 4\nSkipping line 1710: expected 3 fields, saw 4\nSkipping line 2699: expected 3 fields, saw 4\nSkipping line 2728: expected 3 fields, saw 4\nSkipping line 3000: expected 3 fields, saw 4\n'
b'Skipping line 4015: expected 3 fields, saw 4\nSkipping line 6118: expected 3 fields, saw 4\nSkipping line 6354: expected 3 fields, saw 4\nSkipping line 6429: expected 3 fields, saw 4\nSkipping line 6528: expected 3 fields, saw 4\nSkipping line 6930: expected 3 fields, saw 4\nSkipping line 6944: expected 3 fields, saw 4\nSkipping line 6948: expected 3 fields, saw 4\nSkipping line 6949: expected 3 

In [3]:
tweets

,datetime,text,source
0,,,
586266687948881921,Thu Apr 09 20:37:25 +0000 2015,drugs need careful monitoring for expiry dates...,cbchealth
586266687017771008,Thu Apr 09 20:37:25 +0000 2015,sabra hummus recalled in u.s. <EOS>,cbchealth
586266685495214080,Thu Apr 09 20:37:24 +0000 2015,u.s. sperm bank sued by canadian couple didn '...,cbchealth
586226316820623360,Thu Apr 09 17:57:00 +0000 2015,manitoba pharmacists want clampdown on tylenol...,cbchealth
586164344452354048,Thu Apr 09 13:50:44 +0000 2015,mom of 7 ' spooked ' by vaccinations reverses ...,cbchealth
586164343701508096,Thu Apr 09 13:50:44 +0000 2015,hamilton police send mental health pros to the...,cbchealth
586143262810845184,Thu Apr 09 12:26:58 +0000 2015,wind turbine noise linked to only 1 health iss...,cbchealth
586104681849425921,Thu Apr 09 09:53:40 +0000 2015,' wild west ' of e - cigarettes sparks debate...,cbchealth
586104681136398337,Thu Apr 09 09:53:39 +0000 2015,dementia patients sold unproven ' brainwave op...,cbchealth


In [4]:
raw_text = ' '.join(tweets.text)

# Going by Char

In [5]:
# chars = sorted(list(set(raw_text)))
# char_to_int = dict((c, i) for i, c in enumerate(chars))
# n_chars = len(raw_text)
# n_vocab = len(chars)

In [6]:
# seq_length = 100
# dataX =  
# dataY = []
# for i in range(0, n_chars - seq_length, 1):
#     seq_in = raw_text[i:i + seq_length]
#     seq_out = raw_text[i + seq_length]
#     dataX.append([char_to_int[char] for char in seq_in])
#     dataY.append(char_to_int[seq_out])
#     if i % 100000 == 0: print(i)
# n_patterns = len(dataX)

0
100000
200000
300000
400000
500000
600000
700000


In [7]:
# n_chars

794984

In [8]:
# x = np.reshape(dataX, (n_patterns, seq_length, 1))
# x = x / float(n_vocab)
# y = np_utils.to_categorical(dataY)

In [9]:
# model = Sequential()
# model.add(LSTM(256, input_shape=(x.shape[1], x.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[i], activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam')

IndexError: tuple index out of range

# By Word

In [24]:
word_split = raw_text.split()

word_counter = Counter(word_split)

word_split = [word if word_counter[word] > 20 else '<UNK>' for word in word_split]
words = sorted(list(set(word_split)))

word_to_int = dict((w, i) for i, w in enumerate(words))
n_words = len(word_split)
n_vocab = len(words)

In [8]:
# Prepare the dataset of input to output pairs encoded as integers
seq_length = 20

dataX = []
dataY = []
for i in range(0, n_words - seq_length, 1):
    seq_in = word_split[i:i + seq_length]
    seq_out = word_split[i + seq_length]
    dataX.append([word_to_int[word] for word in seq_in])
    dataY.append(word_to_int[seq_out])
    if i % 100000 == 0: print(i)
n_patterns = len(dataX)

0
100000


In [9]:
x = np.reshape(dataX, (n_patterns, seq_length, 1))
x = x / float(n_vocab)
y = np_utils.to_categorical(dataY)
x.shape

(140655, 20, 1)

In [10]:
model_words = Sequential()
model_words.add(LSTM(256, input_shape=(x.shape[1], x.shape[2]), return_sequences=True))
model_words.add(Dropout(0.2))
model_words.add(LSTM(256))
model_words.add(Dropout(0.2))
model_words.add(Dense(y.shape[1], activation='softmax'))
model_words.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
filepath = 'weights-lstm-improvement-{epoch:02d}-{loss:.4f}-bigger.hdfs'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model_words.fit(x, y, epochs = 50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
140655/140655 [==============================] - 232s 2ms/step - loss: 4.5334

Epoch 00001: loss improved from inf to 4.53339, saving model to weights-lstm-improvement-01-4.5334-bigger.hdfs
Epoch 2/50
140655/140655 [==============================] - 211s 1ms/step - loss: 4.4463

Epoch 00002: loss improved from 4.53339 to 4.44632, saving model to weights-lstm-improvement-02-4.4463-bigger.hdfs
Epoch 3/50
140655/140655 [==============================] - 211s 2ms/step - loss: 4.3217

Epoch 00003: loss improved from 4.44632 to 4.32166, saving model to weights-lstm-improvement-03-4.3217-bigger.hdfs
Epoch 4/50
140655/140655 [==============================] - 212s 2ms/step - loss: 4.2418

Epoch 00004: loss improved from 4.32166 to 4.24183, saving model to weights-lstm-improvement-04-4.2418-bigger.hdfs
Epoch 5/50
140655/140655 [==============================] - 4037s 29ms/step - loss: 4.1982

Epoch 00005: loss improved from 4.24183 to 4.19825, saving model to weights-lstm-improvement

Epoch 41/50
140655/140655 [==============================] - 210s 1ms/step - loss: 3.1609

Epoch 00041: loss improved from 3.17918 to 3.16092, saving model to weights-lstm-improvement-41-3.1609-bigger.hdfs
Epoch 42/50
140655/140655 [==============================] - 210s 1ms/step - loss: 3.1484

Epoch 00042: loss improved from 3.16092 to 3.14843, saving model to weights-lstm-improvement-42-3.1484-bigger.hdfs
Epoch 43/50
140655/140655 [==============================] - 211s 2ms/step - loss: 3.1280

Epoch 00043: loss improved from 3.14843 to 3.12800, saving model to weights-lstm-improvement-43-3.1280-bigger.hdfs
Epoch 44/50
140655/140655 [==============================] - 212s 2ms/step - loss: 3.1103

Epoch 00044: loss improved from 3.12800 to 3.11029, saving model to weights-lstm-improvement-44-3.1103-bigger.hdfs
Epoch 45/50
140655/140655 [==============================] - 212s 2ms/step - loss: 3.0967

Epoch 00045: loss improved from 3.11029 to 3.09665, saving model to weights-lstm-impr

In [55]:
import sys
int_to_word = dict((i, w) for i, w in enumerate(words))

model_words = Sequential()
model_words.add(LSTM(256, input_shape=(x.shape[1], x.shape[2]), return_sequences=True))
model_words.add(Dropout(0.2))
model_words.add(LSTM(256))
model_words.add(Dropout(0.2))
model_words.add(Dense(y.shape[1], activation='softmax'))
filename = "weights-lstm-improvement-50-3.0268-bigger.hdfs"
model_words.load_weights(filename)
model_words.compile(loss='categorical_crossentropy', optimizer='adam')

start = np.random.randint(0, len(dataX) - 1)
pattern = dataX[start]
print('Seed:')
print('\"', ' '.join([int_to_word[value] for value in pattern]), '\"')

for i in range(150):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model_words.predict(x, verbose=0)
    index = np.argsort(prediction)
    result = int_to_word[index[0, 1]]
    seq_in = [int_to_word[value] for value in pattern]
    sys.stdout.write(result + ' ')
    pattern.append(index[0, 1])
    pattern = pattern[1:len(pattern)]

print('\nDone.')

Seed:
" , chemo 4 financial 4 low costly 30 he 5 device 2nd 4 4 2nd 4 4 7 aids advocates "
found kills maker 000 gay dead ford growth facebook ford maker kills doing devices diagnosed kills kills african devices concern kills 000 atlanta facebook maker cure african bkesling against kills gop animals 000 malaria doing drugs. brian against ford kills maker cure cure kills ford 000 atlanta facebook maker kills cure cure kills ford 000 african facebook ford ford kills cure cure kills ford electronic african bkesling against kills gop animals devices maker kills maker devices cure maker ford maker cure cure ford maker 000 bkesling facebook kills ford maker 000 bkesling facebook ford ford maker cure 000 bkesling maker maker 000 african bkesling found found kills kills cure cure maker kills devices animals gay maker maker cure cure ford facebook ford 000 bkesling devices kills kills cure devices ford maker 000 atlanta dead ford maker maker african african malaria maker 000 billion kills faceb

In [50]:
x = np.reshape(pattern, (1, len(pattern), 1))
x = x / float(n_vocab)
prediction = model_words.predict(x, verbose=0)
index = np.argsort(prediction)

int_to_word[index[0, 3]]

'devices'